<a href="https://colab.research.google.com/github/hVie1314/olama_google_colab/blob/main/Llama_3_1_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!pip -qq install langchain langchain-core langchain-community
!pip install colab-xterm

In [ ]:
%load_ext colabxterm
%xterm

# TODO: Run this line inside xterm
# ollama serve & ollama pull llama3.1 & ollama pull nomic-embed-text

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.1")

response = llm.invoke("What is the meaning of life?")
print(response)

The ultimate question!

The meaning of life is a complex and subjective concept that has been debated by philosophers, theologians, scientists, and thinkers for centuries. There's no one definitive answer, but I'll try to provide some perspectives and ideas.

**Philosophical views:**

1. **Existentialism**: This perspective suggests that life has no inherent meaning; it's up to each individual to create their own purpose.
2. **Hedonism**: The pursuit of happiness and pleasure is considered the ultimate goal, with life being a series of experiences to be enjoyed.
3. **Utilitarianism**: The greatest good for the greatest number is the guiding principle, with actions motivated by a desire to maximize overall well-being.

**Religious perspectives:**

1. **Christianity**: Life's meaning is found in serving God and fulfilling one's spiritual purpose.
2. **Buddhism**: The aim is to achieve enlightenment and end suffering through self-realization and understanding of the universe.
3. **Hinduis

In [ ]:
!ollama pull llama3.1 & ollama pull nomic-embed-text

In [ ]:
!pip install gradio ollama chromadb

In [ ]:
import ollama
from bs4 import BeautifulSoup as bs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

# Load the data from the web URL
url = 'https://vnexpress.net/de-xuat-thoi-gian-lam-viec-du-ngan-de-nam-nu-co-thoi-gian-tim-ban-doi-4778404.html'
loader = WebBaseLoader(url)
docs = loader.load()

# or load the text from pdf
# path = "../data/Understanding_Climate_Change.pdf"
# loader = PyPDFLoader(path)
# documents = loader.load()

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text") # or use (model = "llama3")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Define the function to call the Ollama Llama3 model
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama3.1', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG setup
retriever = vectorstore.as_retriever() # as_retriever(search_kwargs={"k": 2})

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)



In [ ]:
# Giao diện nhập liệu form bên trong Colab
import gradio as gr
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
  outputs="text",
  title="RAG with Llama3.1",
  description="Ask questions about the provided context",
)

# Launch the Gradio app
iface.launch()

In [ ]:
print(get_important_facts("suggest questions for the document in Vietnamese"))

Dưới đây là một số câu hỏi có thể đặt cho văn bản này:

1. **Thời gian làm việc đủ ngắn để nam nữ tìm bạn đời?**: Câu hỏi này liên quan đến đề xuất của GS.TS Nguyễn Thiện Nhân về thời gian làm việc đủ ngắn (8 giờ lao động/ngày, 40 giờ/tuần) để người lao động có thể tìm bạn đời và chăm sóc gia đình.

2. **Lý do kết hôn muộn hoặc không kết hôn?**: Câu hỏi này liên quan đến lý do khiến nhiều người lựa chọn cuộc sống độc thân hoặc kết hôn muộn, bao gồm họ bận rộn công việc, gặp khó khăn về tài chính...

3. **Tổng tỷ suất sinh của Việt Nam sẽ tiếp tục giảm sâu hơn và duy trì lâu dài?**: Câu hỏi này liên quan đến dự báo của GS.TS Nguyễn Thiện Nhân về tổng tỷ suất sinh của Việt Nam.

4. **Đề xuất bỏ quy định mỗi cặp vợ chồng chỉ sinh 1-2 con?**: Câu hỏi này liên quan đến đề xuất của ông Lê Thanh Dũng, Cục trưởng Dân số (Bộ Y tế) về việc trao quyền quyết định số con cho vợ chồng.

5. **Thời gian làm việc của người lao động ở Việt Nam hiện nay là bao nhiêu?**: Câu hỏi này liên quan đến thông ti